In [ ]:
import os 
import cv2
import numpy as np
from tqdm import tqdm


REBUILD = False

class IMAGES():
	def __init__(self,SRC="./data/HR/",DEST="./data/LR/"):
		self.src = SRC;
		self.dest = DEST;
		self.ref = [];
		self.target = [];
		self.save_data = False

	def make_low_res_data(self):
		for file in tqdm(os.listdir(self.src)):
			sourcepath = os.path.join(self.src,file)
			img = cv2.imread(sourcepath)
			blurImg = cv2.blur(img,(10,10))
			destpath = os.path.join(self.dest,file)
			cv2.imwrite(destpath,blurImg)

	def save_target_data_to_numpy(self):
		for file in tqdm(os.listdir(self.src)):
			sourcepath = os.path.join(self.src,file)
			img = cv2.imread(sourcepath,cv2.COLOR_BGR2YCrCb)
			img = cv2.resize(img,(100,100));
			self.target.append(np.array(img));

		np.random.shuffle(self.target);
		if self.save_data:
			np.save("target_data.npy",self.target);

	def save_ref_data_to_numpy(self):
		for file in tqdm(os.listdir(self.dest)):
			sourcepath = os.path.join(self.dest,file)
			img = cv2.imread(sourcepath,cv2.COLOR_BGR2YCrCb)
			img = cv2.resize(img,(100,100));
			self.ref.append(np.array(img));
		
		np.random.shuffle(self.ref);
		if self.save_data:
			np.save("ref_data.npy",self.ref);




data_build = IMAGES()
if REBUILD:
	data_build.make_low_res_data()
	
data_build.save_target_data_to_numpy()
data_build.save_ref_data_to_numpy()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
	def __init__(self):
		super().__init__()	
		self.conv1 = nn.Conv2d(1,32,9)
		self.conv2 = nn.Conv2d(32,64,9)
		self.conv3 = nn.Conv2d(64,100,9)

	def forward(self,x):
		x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
		x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
		x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))

		return x


net = Net()


In [ ]:
import torch.optim as optim



def modcrop(img,scale):
	tmpsz = img.shape
	sz = tmpsz[0:2]
	sz = sz - np.mod(sz,scale)
	img = img[0:sz[0],1:sz[1]]
	return img



optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_function = nn.MSELoss()


BATCH_SIZE = 10
EPOCHS = 1

X = torch.Tensor([i for i in data_build.ref]).view(-1,100,100)
X = X/255.0
y = torch.Tensor([i for i in data_build.target]).view(-1,100,100)


In [ ]:
for epoch in range(EPOCHS):
	for i in tqdm(range(0,len(data_build.ref),BATCH_SIZE)):
		batch_x =  X[i:i+BATCH_SIZE].view(-1,1,100,100)
		batch_y = y[i:i+BATCH_SIZE].view(-1,100,100)


		net.zero_grad()
		outputs = net(batch_x)
		print(outputs)
		loss = loss_function(outputs,batch_y)
		loss.backward()
		optimizer.step()
